In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
import torch
import re

from src.main import setup_torch, get_corpus
from src.utils import get_latest_model_file
from src.winograd_schema_challenge import (
    analyse_single_wsc, find_missing_wsc_words_in_corpus_vocab, winograd_test
)

from src.wsc_parser import generate_df, generate_json, generate_full_sentences

### Sentences with issues:

In [3]:
still_in_english = [
    "It was a summer afternoon, and the dog was sitting in the middle of the lawn. After a while, it got up and moved to a spot under the tree, because the spot under the tree was hot.",
    "It was a summer afternoon, and the dog was sitting in the middle of the lawn. After a while, it got up and moved to a spot under the tree, because the dog was cooler.",
    "I couldn't put the pot on the shelf because the shelf was too tall.",
    "I couldn't put the pot on the shelf because the pot was too high.",
    "There is a pillar between me and the stage, and I can't see around the stage.",
    "There is a pillar between me and the stage, and I can't see the pillar.",
    "They broadcast an announcement, but a subway came into the station and I couldn't hear the subway.",
    "They broadcast an announcement, but a subway came into the station and I couldn't hear over the announcement.",
]

In [4]:
df = generate_df(still_in_english)
df['correct_sentence'], df['incorrect_sentence'] = zip(*df.apply(generate_full_sentences, axis=1))

In [5]:
df['correct_sentence'].replace({'alice': 'Alice'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'alice': 'Alice'}, inplace=True, regex=True)
df['correct_sentence'].replace({'fred': 'Fred'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'fred': 'Fred'}, inplace=True, regex=True)
df['correct_sentence'].replace({'mary': 'Mary'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'mary': 'Mary'}, inplace=True, regex=True)
df['correct_sentence'].replace({'anne': 'Anne'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'anne': 'Anne'}, inplace=True, regex=True)
df['correct_sentence'].replace({'jim': 'Jim'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'jim': 'Jim'}, inplace=True, regex=True)
df['correct_sentence'].replace({'O golpista': 'o golpista'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'O golpista': 'o golpista'}, inplace=True, regex=True)
df['correct_sentence'].replace({'desenho da tina': 'desenho da Tina'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'desenho da tina': 'desenho da Tina'}, inplace=True, regex=True)
df['correct_sentence'].replace({'desenho do sam': 'desenho do Sam'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'desenho do sam': 'desenho do Sam'}, inplace=True, regex=True)
df['correct_sentence'].replace({' Os pais da Amy': ' os pais da Amy'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({' Os pais da Amy': ' os pais da Amy'}, inplace=True, regex=True)
df['correct_sentence'].replace({' O tio do Joe': ' o tio do Joe'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({' O tio do Joe': ' o tio do Joe'}, inplace=True, regex=True)
df['correct_sentence'].replace({'joe': 'Joe'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'joe': 'Joe'}, inplace=True, regex=True)
df['correct_sentence'].replace({' A Dra. Adams': ' a Dra. Adams'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({' A Dra. Adams': ' a Dra. Adams'}, inplace=True, regex=True)
df['correct_sentence'].replace({' A Kate': ' a kate'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({' A Kate': ' a kate'}, inplace=True, regex=True)
df['correct_sentence'].replace({'madonna': 'Madonna'}, inplace=True, regex=True)
df['incorrect_sentence'].replace({'madonna': 'Madonna'}, inplace=True, regex=True)

In [6]:
# get list of words that are currently capitalized, and are not first word at sentence or after period mark
capitalized_words = []

def get_capitalized_words(schema):
    global capitalized_words
    words = re.findall(r'(?<!^)(?<!\. )[A-Z][a-z]+', schema)
    
    capitalized_words += words

df.schema.apply(get_capitalized_words)
df.correct_sentence.apply(get_capitalized_words)
df.incorrect_sentence.apply(get_capitalized_words)

capitalized_words += ['Índico', 'GrWQWu8JyC', 'Ovídio', 'Frank', 'Jane', 'Sid'] 
capitalized_words = list(set(capitalized_words) - set(['Gr', 'Jy', 'Wu', 'Ov', 'Eu']))
# Eu primeiro!
capitalized_words.sort()

with open('data/processed/portuguese_capitalized_words.txt', 'w') as outfile:
    for item in capitalized_words:
        outfile.write("%s\n" % item)